<a href="https://colab.research.google.com/github/mateuszklimczyk97/data-science-bootcamp/blob/main/06_uczenie_maszynowe/02_metryki_klasyfikacja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Metryki - Problem klasyfikacji
(ocena kredytowa, czy klient popełni daną szkodę jako [ubezpieczyciel], klasyfikacja zdjęć [koty,psy,konie])

### Import bibliotek

In [3]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

### Metryki - Klasyfikacja binarna
#### Accuracy - Dokładność klasyfikacji

In [13]:
y_true = np.array([1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1]) #macierz prawdziwych wartości
y_pred = np.array([0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1]) #macierz predykcji  

In [14]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true,y_pred)

0.8

In [15]:
results = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})
results

,y_true,y_pred
0,1,0
1,0,0
2,1,1
3,1,1
4,0,0
5,1,1
6,1,0
7,0,0
8,1,1
9,0,1


### Wizualizacja predykcji

In [16]:
# sortowanie
results = results.sort_values(by='y_true')
results = results.reset_index(drop=True)
results['sample'] = results.index+1
results

,y_true,y_pred,sample
0,0,0,1
1,0,0,2
2,0,0,3
3,0,0,4
4,0,0,5
5,0,0,6
6,0,0,7
7,0,0,8
8,0,1,9
9,0,1,10


In [17]:
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=results['sample'], y=results['y_true'], mode='markers', name='y_true'), row=1,col=1)
fig.add_trace(go.Scatter(x=results['sample'], y=results['y_pred'], mode='markers', name='y_ored'), row=2,col=1)
fig.update_layout(width=900,height=500, title='Klasyfikator binarny')
fig.show()

### Macierz konfuzji/pomylek

In [18]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true,y_pred)
cm

array([[ 8,  2],
       [ 4, 16]])

In [19]:
import plotly.figure_factory as ff

def plot_confusion_matrix(cm):
  cm = cm[::-1]
  cm = pd.DataFrame(cm, columns=['pred_0','pred_1'], index=['true_1','true_2'])

  fig = ff.create_annotated_heatmap(z=cm.values, x=list(cm.columns), y=list(cm.index), colorscale='ice', showscale=True, reversescale=True)
  fig.update_layout(width=400,height=400,title='Macierz konfuzji', font_size=16)
  fig.show()

plot_confusion_matrix(cm)

In [20]:
tn, fp, fn, tp = cm.ravel() #jednowymiarowa tablica
print(f'TN - True Negative: {tn}')
print(f'FP - False Positive: {fp}')
print(f'FN - False Negative: {fn}')
print(f'TP - True Positive: {tp}')

TN - True Negative: 8
FP - False Positive: 2
FN - False Negative: 4
TP - True Positive: 16


In [21]:
#Bład pierwszego rodzaju - Type I error - False Positive Rate
fpr = fp / (fp+tn)
fpr

0.2

In [22]:
#Bład drugiego rodzaju - Type II error - False Negative Rate
fnr = fn / (fn+tp)
fnr

0.2

In [23]:
#Prezycja - Precision - ile obserwacji przewidywanych jako pozytywne są w rzeczywistosci pozytywne
precision = tp / (tp+fp)
precision

0.8888888888888888

In [24]:
#Recall - jak wiele obserwacji z wszystkich pozytywnych sklasyfikowalismy jako pozytywne
recall = tp / (tp+fn)
recall

0.8

### Krzywa ROC

In [25]:
from sklearn.metrics import roc_curve

fpr, tpr, thresh = roc_curve(y_true,y_pred, pos_label=1)
roc = pd.DataFrame({'fpr': fpr, 'tpr': tpr})
roc

,fpr,tpr
0,0.0,0.0
1,0.2,0.8
2,1.0,1.0


In [26]:
from sklearn.metrics import roc_curve

def plot_roc_curve(y_true,y_pred):
  fpr, tpr, thresh = roc_curve(y_true,y_pred, pos_label=1)

  fig = go.Figure(data=[go.Scatter(x=roc['fpr'], y=roc['tpr'], line_color='red', name='Kzywa ROC'),
                        go.Scatter(x=[0,1],y=[0,1],mode='lines',line_dash='dash',line_color='navy')],
                  layout=go.Layout(xaxis_title='False Positive Rate',
                                   yaxis_title='True Positive Rate',
                                   title='Krzywa ROC',
                                   showlegend=False,
                                   width=800,
                                   height=400))
  fig.show()

plot_roc_curve(y_true,y_pred)   

In [ ]:
#wybieramy model ktory ma pod krzywą najwieksze pole (linia przerywana - wartosci losowe)

## Metryki - Klasyfikacja wieloklasowa

### Accuracy - Dokładność klasyfikacji

In [27]:
y_true = np.array([1, 0, 1, 2, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 2, 1, 1, 2, 2, 1, 0, 1, 1, 0, 2, 1, 1, 2, 2])
y_pred = np.array([0, 0, 1, 2, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 2, 1, 2, 1, 2, 1, 0, 2, 1, 0, 1, 1, 1, 2, 2])

accuracy_score(y_true, y_pred)

0.7241379310344828

### Macierz konfuzji/pomyłek

In [28]:
cm = confusion_matrix(y_true, y_pred)
cm

array([[ 6,  1,  0],
       [ 3, 10,  2],
       [ 0,  2,  5]])

In [29]:
def plot_confusion_matrix(cm):
  cm = cm[::-1]
  cm = pd.DataFrame(cm, columns=['pred_0','pred_1','pred_2'], index=['true_2','true_1','true_0'])

  fig = ff.create_annotated_heatmap(z=cm.values, x=list(cm.columns), y=list(cm.index), colorscale='ice', showscale=True, reversescale=True)
  fig.update_layout(width=500,height=500,title='Macierz konfuzji', font_size=16)
  fig.show()

plot_confusion_matrix(cm)

### Raport klasyfikacji

In [32]:
from sklearn.metrics import classification_report

print(classification_report(y_true,y_pred))
#f1-score (srednia harmoniczna[precision, recall])

              precision    recall  f1-score   support

           0       0.67      0.86      0.75         7
           1       0.77      0.67      0.71        15
           2       0.71      0.71      0.71         7

    accuracy                           0.72        29
   macro avg       0.72      0.75      0.73        29
weighted avg       0.73      0.72      0.72        29



In [31]:
print(classification_report(y_true,y_pred, target_names= ['label_1','label_2','label_3'])) #nadanie klasom nazw

              precision    recall  f1-score   support

     label_1       0.67      0.86      0.75         7
     label_2       0.77      0.67      0.71        15
     label_3       0.71      0.71      0.71         7

    accuracy                           0.72        29
   macro avg       0.72      0.75      0.73        29
weighted avg       0.73      0.72      0.72        29

